In [1]:
import json
from tqdm import tqdm

path = "/root/personal/datasets/PM4Bench/MDUR_multilingual_for_qwenvl_base64.jsonl"

n_total = 0
n_single = 0
n_multi = 0
multi_examples = []

with open(path, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="Checking samples"):
        sample = json.loads(line)
        n_total += 1
        n_images = len(sample.get("images", []))
        if n_images == 1:
            n_single += 1
        elif n_images > 1:
            n_multi += 1
            multi_examples.append({
                "id": sample["id"],
                "num_images": n_images,
                "langs": list(sample["questions"].keys())
            })

print(f"\n📊 Total samples: {n_total}")
print(f"✅ Single-image samples: {n_single}")
print(f"⚠️ Multi-image samples: {n_multi}")

if multi_examples:
    print("\n🔍 Example multi-image entries:")
    for e in multi_examples[:5]:
        print(f" - ID: {e['id']}, num_images: {e['num_images']}, langs: {e['langs']}")


Checking samples: 1491it [00:03, 425.08it/s]


📊 Total samples: 1491
✅ Single-image samples: 1370
⚠️ Multi-image samples: 121

🔍 Example multi-image entries:
 - ID: 709, num_images: 2, langs: ['AR', 'VI', 'CS', 'ZH', 'RU', 'TH', 'KO', 'HU', 'EN', 'SR']
 - ID: 1066, num_images: 2, langs: ['AR', 'VI', 'CS', 'ZH', 'RU', 'TH', 'KO', 'HU', 'EN', 'SR']
 - ID: 242, num_images: 4, langs: ['AR', 'VI', 'CS', 'ZH', 'RU', 'TH', 'KO', 'HU', 'EN', 'SR']
 - ID: 676, num_images: 2, langs: ['AR', 'VI', 'CS', 'ZH', 'RU', 'TH', 'KO', 'HU', 'EN', 'SR']
 - ID: 956, num_images: 2, langs: ['AR', 'VI', 'CS', 'ZH', 'RU', 'TH', 'KO', 'HU', 'EN', 'SR']


---

In [1]:
"""
Qwen2.5-VL: 提取 Hidden States（从 JSONL base64 图像 + 多语言文本）
-----------------------------------------------------------
单样本演示版，用于验证前向输出并保存所有层 hidden state。
"""

import json
import base64
from io import BytesIO
from PIL import Image
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info


# ============================================================
# 1️⃣ 加载模型与处理器（推荐：fp16 + eager，避免 kernel 崩溃）
# ============================================================
model_path = "/root/personal/hf_models/Qwen-VL-2.5/7B-Instruct"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.float16,       # ✅ 稳定
    attn_implementation="eager",     
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(model_path)


# ============================================================
# 2️⃣ 从你的 JSONL 数据加载一条样本
# ============================================================
data_path = "/root/personal/datasets/PM4Bench/MDUR_multilingual_for_qwenvl_base64.jsonl"
lang = "EN"  # 可切换 "ZH", "AR", "VI", 等

# 读取一个单图样本
with open(data_path, "r", encoding="utf-8") as f:
    for line in f:
        sample = json.loads(line)
        if len(sample.get("images", [])) == 1 and lang in sample["questions"]:
            break  # 找到第一个符合条件的样本

# 构造输入
question = sample["questions"][lang].replace("<image 1>", "")
options = "\n".join([f"{chr(65+i)}. {opt}" for i, opt in enumerate(sample["options"])])
text = question + "\n" + options

# 解码 base64 图片
img = Image.open(BytesIO(base64.b64decode(sample["images"][0])))

# 构建对话输入
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": img},
            {"type": "text", "text": text},
        ],
    }
]

# 构造模型输入
text_prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text_prompt],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")


# ============================================================
# 3️⃣ 前向计算并提取 hidden states
# ============================================================
with torch.inference_mode():
    outputs = model(**inputs, output_hidden_states=True, use_cache=False)

hidden_states = outputs.hidden_states  # list of [B, seq_len, dim]

print("====== Hidden States Info ======")
print(f"总层数: {len(hidden_states)} (含 embedding 层)")
print(f"最后一层形状: {hidden_states[-1].shape}")  # [1, seq_len, dim]


# ============================================================
# 4️⃣ 保存 hidden states 到文件
# ============================================================
save_path = f"hidden_state_{sample['id']}_{lang}.pt"
torch.save(
    [h[0].cpu().to(torch.float16) for h in hidden_states],  # 保存每层一个 tensor
    save_path,
)

print(f"✅ 已保存 hidden states 到 {save_path}")


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


RuntimeError: GET was unable to find an engine to execute this computation